# Learning Agent - AI Tutor

Welcome to the Learning Agent workshop! This notebook demonstrates how to build an AI Tutor that provides personalized learning experiences, adapts to user comprehension, and dynamically creates learning paths.

## Vision & Goals

This Learning Agent fundamentally changes how learning is conducted by:
- **Personalized Interaction**: One-on-one AI teacher that adapts to each learner
- **Dynamic Content Delivery**: Presents information based on dialogue and comprehension
- **Reinforcement Learning**: Ensures concepts are understood before progressing
- **Custom Learning Paths**: Tailored recommendations based on employee data

## What You'll Learn

By the end of this notebook, you'll understand:

1. **Personalized AI Tutoring** - How to create adaptive learning experiences
2. **Learning Path Generation** - Dynamic curriculum creation based on user needs
3. **Comprehension Assessment** - Real-time evaluation of user understanding
4. **Knowledge Reinforcement** - Ensuring concepts are mastered before progression
5. **Employee-Focused Learning** - Integration with workplace goals and aspirations

## Key Features

Our AI Tutor will:
- Assess current knowledge and learning style
- Create personalized learning paths
- Provide interactive Q&A on course material
- Adapt teaching methods based on comprehension
- Track progress and adjust accordingly
- Integrate with employee career goals

Let's build an intelligent learning companion! 🎓

## Import Required Packages

We'll use the same Semantic Kernel foundation as other workshops, with additional focus on educational interactions and learning path management.

### Core Libraries
- `json` - For handling structured learning data
- `datetime` - For tracking learning progress and schedules
- `typing.Annotated` - For clear function parameter descriptions

### Semantic Kernel Components
- `ChatCompletionAgent` - Our AI Tutor agent
- `ChatHistoryAgentThread` - Maintains learning conversation context
- `AzureChatCompletion` - Azure OpenAI integration
- `kernel_function` - For creating educational tools and assessments

In [ ]:
# Install required packages from requirements.txt
%pip install -r requirements.txt

In [ ]:
import os 
import json
from datetime import datetime, timedelta
from typing import Annotated, Dict, List
from azure.identity import DefaultAzureCredential

from dotenv import load_dotenv

from IPython.display import display, HTML

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## Creating the Learning Management Plugin

This plugin simulates an employee learning management system that provides:

1. **Employee Profile Data** - Role, department, career goals, performance metrics
2. **Learning Path Generation** - Custom curricula based on employee data
3. **Knowledge Assessment** - Interactive quizzes and comprehension checks
4. **Progress Tracking** - Learning analytics and milestone tracking
5. **Content Library** - Course materials and resources

### Real-World Integration

In a production environment, this plugin would integrate with:
- HR systems for employee data
- Learning Management Systems (LMS)
- Performance management platforms
- Career development tools
- Content management systems

In [ ]:
class LearningManagementPlugin:
    """AI Tutor Plugin for personalized learning experiences."""

    def __init__(self):
        # Sample employee data (in production, this would come from HR systems)
        self.employee_profiles = {
            "john_doe": {
                "name": "John Doe",
                "role": "Software Engineer",
                "department": "Engineering",
                "location": "Seattle, WA",
                "career_goals": ["Senior Engineer", "Tech Lead", "Solution Architect"],
                "target_roles": ["Senior Software Engineer", "Engineering Manager"],
                "performance_goals": ["Improve system design skills", "Learn cloud architecture", "Enhance team leadership"],
                "development_goals": ["Azure certification", "Machine learning fundamentals", "Agile methodologies"],
                "current_level": "Mid-level",
                "learning_style": "Visual and hands-on",
                "completed_courses": ["Python Fundamentals", "Git Basics", "API Design"]
            },
            "sarah_smith": {
                "name": "Sarah Smith", 
                "role": "Data Analyst",
                "department": "Data Science",
                "location": "Austin, TX",
                "career_goals": ["Data Scientist", "ML Engineer", "Analytics Manager"],
                "target_roles": ["Senior Data Scientist", "ML Engineering Lead"],
                "performance_goals": ["Advanced statistical analysis", "ML model deployment", "Data storytelling"],
                "development_goals": ["Deep learning", "MLOps", "Executive communication"],
                "current_level": "Junior",
                "learning_style": "Theory-first with practical examples",
                "completed_courses": ["Statistics 101", "SQL Fundamentals", "Data Visualization"]
            }
        }

        # Learning content library
        self.learning_content = {
            "Azure Fundamentals": {
                "description": "Introduction to cloud computing and Azure services",
                "duration": "4 weeks",
                "difficulty": "Beginner", 
                "prerequisites": ["Basic IT knowledge"],
                "modules": ["Cloud Concepts", "Azure Services", "Security", "Pricing"]
            },
            "Machine Learning Basics": {
                "description": "Foundational concepts in machine learning",
                "duration": "6 weeks",
                "difficulty": "Intermediate",
                "prerequisites": ["Python programming", "Statistics"],
                "modules": ["ML Algorithms", "Data Preprocessing", "Model Evaluation", "Ethics"]
            },
            "Leadership Skills": {
                "description": "Essential skills for technical leadership",
                "duration": "3 weeks", 
                "difficulty": "All levels",
                "prerequisites": ["None"],
                "modules": ["Communication", "Team Management", "Decision Making", "Conflict Resolution"]
            },
            "System Design": {
                "description": "Designing scalable software systems",
                "duration": "8 weeks",
                "difficulty": "Advanced",
                "prerequisites": ["Software Engineering experience", "Database knowledge"],
                "modules": ["Architecture Patterns", "Scalability", "Database Design", "Microservices"]
            }
        }

    @kernel_function(description="Get employee profile information for personalized learning.")
    def get_employee_profile(
        self, employee_id: Annotated[str, "The employee ID to look up (e.g., 'john_doe', 'sarah_smith')"]
    ) -> Annotated[str, "Returns employee profile with career goals and learning preferences"]:
        
        employee = self.employee_profiles.get(employee_id.lower())
        if not employee:
            return f"Employee profile not found for ID: {employee_id}. Available IDs: {', '.join(self.employee_profiles.keys())}"
        
        profile_info = f"""
Employee Profile: {employee['name']}
Role: {employee['role']} 
Department: {employee['department']}
Location: {employee['location']}
Current Level: {employee['current_level']}
Learning Style: {employee['learning_style']}

Career Goals: {', '.join(employee['career_goals'])}
Target Roles: {', '.join(employee['target_roles'])}
Performance Goals: {', '.join(employee['performance_goals'])}
Development Goals: {', '.join(employee['development_goals'])}

Completed Courses: {', '.join(employee['completed_courses'])}
"""
        return profile_info

    @kernel_function(description="Generate a personalized learning path based on employee goals and current skills.")
    def generate_learning_path(
        self, 
        employee_id: Annotated[str, "The employee ID to create a learning path for"],
        focus_area: Annotated[str, "Specific area to focus on (e.g., 'cloud', 'leadership', 'ml', 'engineering')"]
    ) -> Annotated[str, "Returns a customized learning path with recommended courses and timeline"]:
        
        employee = self.employee_profiles.get(employee_id.lower())
        if not employee:
            return f"Employee not found: {employee_id}"

        # Generate recommendations based on focus area and employee profile
        recommendations = []
        
        if "cloud" in focus_area.lower() or "azure" in focus_area.lower():
            recommendations.append("Azure Fundamentals")
            if employee['current_level'] in ['Mid-level', 'Senior']:
                recommendations.append("System Design")
                
        if "leadership" in focus_area.lower() or "management" in focus_area.lower():
            recommendations.append("Leadership Skills")
            
        if "ml" in focus_area.lower() or "machine learning" in focus_area.lower() or "data" in focus_area.lower():
            recommendations.append("Machine Learning Basics")
            if employee['role'] in ['Software Engineer', 'Senior Software Engineer']:
                recommendations.append("System Design")

        if "engineering" in focus_area.lower() or "technical" in focus_area.lower():
            recommendations.append("System Design")
            recommendations.append("Azure Fundamentals")

        # Build learning path
        learning_path = f"""
🎯 Personalized Learning Path for {employee['name']}
Focus Area: {focus_area}
Learning Style: {employee['learning_style']}

Recommended Courses:
"""
        
        total_duration = 0
        for i, course_name in enumerate(recommendations, 1):
            course = self.learning_content.get(course_name, {})
            duration_weeks = int(course.get('duration', '4 weeks').split()[0])
            total_duration += duration_weeks
            
            learning_path += f"""
{i}. {course_name}
   Duration: {course.get('duration', 'N/A')}
   Difficulty: {course.get('difficulty', 'N/A')}
   Prerequisites: {', '.join(course.get('prerequisites', []))}
   Modules: {', '.join(course.get('modules', []))}
"""

        learning_path += f"""
📅 Total Estimated Duration: {total_duration} weeks
🎯 Aligns with career goals: {', '.join(employee['career_goals'])}
💡 Matches learning style: {employee['learning_style']}
"""
        
        return learning_path

    @kernel_function(description="Get detailed course content and materials for a specific course.")
    def get_course_content(
        self, course_name: Annotated[str, "Name of the course to get content for"]
    ) -> Annotated[str, "Returns detailed course information and learning materials"]:
        
        course = self.learning_content.get(course_name)
        if not course:
            available_courses = ', '.join(self.learning_content.keys())
            return f"Course '{course_name}' not found. Available courses: {available_courses}"

        content = f"""
📚 Course: {course_name}

📖 Description: {course['description']}
⏱️ Duration: {course['duration']}
📊 Difficulty: {course['difficulty']}
📋 Prerequisites: {', '.join(course['prerequisites'])}

📚 Learning Modules:
"""
        
        for i, module in enumerate(course['modules'], 1):
            content += f"\n{i}. {module}"
            
        # Add sample learning activities based on course type
        if "Azure" in course_name:
            content += """

🛠️ Hands-on Activities:
- Create Azure account and explore portal
- Deploy virtual machines and web apps
- Configure security and monitoring
- Practice with Azure CLI and PowerShell

📝 Assessment Methods:
- Interactive labs and demos
- Real-world scenario exercises
- Azure certification practice exams
"""
        elif "Machine Learning" in course_name:
            content += """

🛠️ Hands-on Activities:
- Build ML models with Python/Scikit-learn
- Data analysis and visualization projects
- Model evaluation and optimization
- Deploy models to production

📝 Assessment Methods:
- Coding assignments and projects
- Data analysis case studies
- Algorithm implementation exercises
"""
        elif "Leadership" in course_name:
            content += """

🛠️ Interactive Activities:
- Leadership style assessments
- Team simulation exercises
- Communication practice scenarios
- Conflict resolution role-plays

📝 Assessment Methods:
- Self-reflection exercises
- Peer feedback sessions
- Leadership project presentations
"""
        elif "System Design" in course_name:
            content += """

🛠️ Design Exercises:
- Architecture diagram creation
- System scalability planning
- Database schema design
- API design and documentation

📝 Assessment Methods:
- System design interviews
- Architecture documentation
- Code review and optimization
- Performance analysis projects
"""
            
        return content

    @kernel_function(description="Assess user comprehension and provide feedback on learning progress.")
    def assess_comprehension(
        self, 
        topic: Annotated[str, "The learning topic to assess"],
        user_response: Annotated[str, "User's explanation or answer to evaluate"]
    ) -> Annotated[str, "Returns comprehension assessment and learning recommendations"]:
        
        # Simple comprehension assessment based on response content
        response_lower = user_response.lower()
        topic_lower = topic.lower()
        
        # Check for key concepts (this would be more sophisticated in production)
        understanding_score = 0
        feedback = f"📊 Comprehension Assessment for: {topic}\n\n"
        
        if len(user_response.split()) >= 10:  # Detailed response
            understanding_score += 3
            feedback += "✅ Good detail in explanation\n"
        else:
            feedback += "⚠️ Could provide more detailed explanation\n"
            
        # Topic-specific assessments
        if "azure" in topic_lower or "cloud" in topic_lower:
            key_terms = ["service", "virtual", "storage", "compute", "security", "scale"]
            found_terms = sum(1 for term in key_terms if term in response_lower)
            understanding_score += found_terms
            feedback += f"✅ Used {found_terms} key cloud concepts\n"
            
        elif "machine learning" in topic_lower or "ml" in topic_lower:
            key_terms = ["model", "data", "algorithm", "training", "prediction", "accuracy"]
            found_terms = sum(1 for term in key_terms if term in response_lower)
            understanding_score += found_terms
            feedback += f"✅ Used {found_terms} key ML concepts\n"
            
        elif "leadership" in topic_lower:
            key_terms = ["team", "communication", "decision", "management", "goal", "feedback"]
            found_terms = sum(1 for term in key_terms if term in response_lower)
            understanding_score += found_terms
            feedback += f"✅ Used {found_terms} key leadership concepts\n"
            
        elif "system design" in topic_lower:
            key_terms = ["architecture", "scalability", "database", "api", "service", "performance"]
            found_terms = sum(1 for term in key_terms if term in response_lower)
            understanding_score += found_terms
            feedback += f"✅ Used {found_terms} key system design concepts\n"

        # Provide comprehension level and recommendations
        if understanding_score >= 7:
            feedback += """
🎉 Excellent Understanding! 
- You demonstrate strong comprehension of the topic
- Ready to move to more advanced concepts
- Consider peer teaching or project application
"""
        elif understanding_score >= 4:
            feedback += """
👍 Good Understanding
- You grasp the main concepts well
- Review specific areas for deeper insight
- Practice with hands-on exercises recommended
"""
        else:
            feedback += """
📚 Needs Reinforcement
- Let's review the core concepts together
- Additional examples and practice recommended
- Consider alternative learning approaches
"""
            
        return feedback

## Setting up Azure OpenAI Connection

We'll use the same Azure OpenAI setup as other workshops, providing both API key and Azure AD authentication options for flexibility in different environments.

### Learning-Focused Configuration

For educational applications, consider:
- **Consistent Model Behavior**: Use temperature settings that provide reliable, educational responses
- **Content Filtering**: Ensure appropriate content for workplace learning
- **Response Length**: Balance comprehensive explanations with concise delivery
- **Cost Management**: Monitor usage for large-scale educational deployments

In [ ]:
load_dotenv()

# Option 1: Using API Key (recommended for development)
chat_completion_service = AzureChatCompletion(
    deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY")
)

# Option 2: Using Azure AD Authentication (uncomment to use)
# credential = DefaultAzureCredential()

# def get_azure_ad_token():
#     """Function to get Azure AD token for OpenAI."""
#     token = credential.get_token("https://cognitiveservices.azure.com/.default")
#     return token.token

# chat_completion_service = AzureChatCompletion(
#     deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
#     endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
#     api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
#     ad_token=get_azure_ad_token()
# )

## Creating the AI Tutor Agent

Now we'll create our Learning Agent with carefully crafted instructions that enable personalized, adaptive tutoring experiences.

### AI Tutor Personality & Behavior

Our agent instructions define:

1. **Educational Philosophy**: Personalized, adaptive learning approach
2. **Teaching Methods**: Interactive, comprehension-based progression
3. **Assessment Strategy**: Continuous evaluation and feedback
4. **Personalization**: Adapting to learning styles and career goals
5. **Encouragement**: Positive reinforcement and growth mindset

### Key Teaching Principles

- **Adaptive Pacing**: Adjust speed based on comprehension
- **Multiple Modalities**: Support different learning styles
- **Real-World Application**: Connect concepts to career goals
- **Continuous Assessment**: Regular comprehension checks
- **Positive Reinforcement**: Celebrate progress and achievements

### Integration with Employee Data

The agent leverages employee profiles to:
- Customize content to career aspirations
- Align with performance goals
- Respect individual learning preferences
- Track progress against development objectives

In [ ]:
AI_TUTOR_INSTRUCTIONS = """You are an intelligent AI Tutor designed to provide personalized learning experiences for employees. Your mission is to fundamentally change how learning is conducted by creating adaptive, one-on-one educational interactions.

## Core Principles:
1. **Personalization First**: Always consider the learner's role, goals, and learning style
2. **Adaptive Teaching**: Adjust your approach based on comprehension and feedback
3. **Comprehension-Based Progression**: Ensure understanding before moving to new topics
4. **Real-World Application**: Connect learning to career goals and workplace needs
5. **Positive Reinforcement**: Encourage growth mindset and celebrate progress

## Teaching Approach:
- Start by understanding the learner's background and goals
- Use the employee profile to personalize content and examples
- Provide interactive learning with questions and assessments
- Adapt explanations based on learning style (visual, hands-on, theory-first, etc.)
- Regularly check comprehension and adjust pacing accordingly
- Connect concepts to real workplace scenarios and career advancement

## Assessment & Feedback:
- Use Socratic questioning to guide discovery
- Provide immediate, constructive feedback
- Offer multiple explanation approaches if concepts aren't clear
- Celebrate achievements and progress milestones
- Identify knowledge gaps and provide targeted reinforcement

## Learning Path Management:
- Generate custom learning paths based on employee data
- Recommend courses aligned with career goals and role requirements
- Suggest practical applications and projects
- Track progress and adjust recommendations

## Interaction Style:
- Be encouraging, patient, and supportive
- Use clear, jargon-free explanations (unless teaching technical terms)
- Ask follow-up questions to ensure understanding
- Provide examples relevant to the learner's field and goals
- Make learning engaging and interactive

Always begin interactions by learning about the employee and their goals, then adapt your teaching to their specific needs and aspirations."""

# Create the AI Tutor agent
ai_tutor = ChatCompletionAgent(
    service=chat_completion_service,
    name="AI_Tutor",
    instructions=AI_TUTOR_INSTRUCTIONS,
    plugins=[LearningManagementPlugin()],
)

## Running the AI Tutor

Let's test our AI Tutor with realistic workplace learning scenarios. We'll simulate an employee seeking career development guidance and personalized learning recommendations.

### Learning Scenarios

Our test cases demonstrate:

1. **Initial Assessment**: Employee introduction and goal discovery
2. **Learning Path Creation**: Personalized curriculum based on career goals
3. **Interactive Tutoring**: Q&A on specific course content
4. **Comprehension Assessment**: Real-time evaluation and feedback
5. **Adaptive Teaching**: Adjusting approach based on understanding

### What to Observe

Watch how the AI Tutor:
- Adapts its communication style to the learner
- Uses employee data to personalize recommendations
- Provides educational content tailored to career goals
- Assesses comprehension and adjusts accordingly
- Maintains encouraging, supportive interactions

### Educational Value

This approach transforms traditional learning by:
- Eliminating one-size-fits-all course models
- Providing immediate, personalized feedback
- Adapting to individual learning preferences
- Connecting learning to real career advancement
- Ensuring mastery before progression

In [ ]:
# Test scenarios for the AI Tutor
learning_scenarios = [
    "Hi! I'm John Doe, a software engineer looking to advance my career. Can you help me create a learning plan?",
    "I'm interested in cloud technologies, especially Azure. What would you recommend for someone in my role?",
    "Can you explain what Azure Fundamentals covers and how it would help my career goals?",
    "I understand the basics of cloud computing, but can you assess my knowledge? I think cloud computing is about using remote servers to store and process data instead of local computers."
]

async def run_ai_tutor():
    # Initialize conversation thread for continuous learning context
    thread: ChatHistoryAgentThread | None = None

    for scenario in learning_scenarios:
        # Display user input
        html_output = (
            f"<div style='margin-bottom:10px; border-left: 4px solid #2196F3; padding-left: 10px;'>"
            f"<div style='font-weight:bold; color: #2196F3;'>👨‍💼 Learner:</div>"
            f"<div style='margin-left:20px; font-style: italic;'>{scenario}</div></div>"
        )

        # Track response components
        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Function call tracking
        current_function_name = None
        argument_buffer = ""

        # Stream AI Tutor response
        async for response in ai_tutor.invoke_stream(
            messages=scenario,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    # Capture function calls (accessing learning data)
                    if item.function_name:
                        current_function_name = item.function_name

                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Process function results (employee data, learning content)
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass

                        function_calls.append(f"📊 Accessing: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"📋 Data Retrieved:\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    # Collect tutor's educational response
                    full_response.append(item.text)

        # Display function calls (learning data access) in collapsible section
        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#FF9800;'>🔍 Learning Data Access (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#fff3e0; "
                "border:1px solid #ffcc02; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#e65100;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        # Display AI Tutor response with educational styling
        html_output += (
            "<div style='margin-bottom:20px; border-left: 4px solid #4CAF50; padding-left: 10px;'>"
            f"<div style='font-weight:bold; color: #4CAF50;'>🤖 {agent_name or 'AI Tutor'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap; line-height: 1.6;'>{''.join(full_response)}</div></div>"
            "<hr style='border: 1px solid #e0e0e0; margin: 20px 0;'>"
        )

        display(HTML(html_output))

# Run the AI Tutor conversation
await run_ai_tutor()

## 🎓 Congratulations on Building Your AI Tutor!

You've successfully created a sophisticated Learning Agent that demonstrates the future of personalized education! Let's celebrate what you've accomplished:

### 🏆 What You Built

✅ **Intelligent Learning Agent** - AI Tutor with adaptive teaching capabilities  
✅ **Personalization Engine** - Employee-focused learning recommendations  
✅ **Learning Path Generator** - Custom curricula based on career goals  
✅ **Comprehension Assessment** - Real-time evaluation and feedback system  
✅ **Knowledge Management** - Integration with employee data and course content  
✅ **Adaptive Teaching** - Dynamic content delivery based on learning styles  